## 112 신고

In [52]:
import geopandas as gpd
import pandas as pd
import folium
from folium import plugins
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

+ 2018년 6월 2일부터만 존재하는 데이터

In [53]:
call = pd.read_csv('5.112신고.csv')
call

,date,jur_stn,report_sx,inc_info
0,20180603.0,서울서부,남성,분실습득
1,20180603.0,서울송파,여성,폭력
2,20180603.0,서울강서,남성,소음
3,20180603.0,서울영등포,여성,실종(실종아동 등)
4,20180603.0,서울강서,남성,상담문의
...,...,...,...,...
9229106,20210531.0,마산동부,남성,교통사고
9229107,20210531.0,마산중부,남성,가정폭력
9229108,20210531.0,마산중부,불상,화재
9229109,20210531.0,마산동부,남성,기타형사범


In [54]:
call_seoul = call[call['jur_stn'].str.contains('서울', na=False)]
call_sejong = call[call['jur_stn'].str.contains('세종', na=False)]
call_suwon = call[call['jur_stn'].str.contains('수원', na=False)]
call_masan = call[call['jur_stn'].str.contains('마산', na=False)]
call_changwon = call[call['jur_stn'].str.contains('창원', na=False)]
call_jinhae = call[call['jur_stn'].str.contains('진해', na=False)]
call_jinju = call[call['jur_stn'].str.contains('진주', na=False)]

### 결측치 처리

In [55]:
## NA 결측치는 모든 시에서 2% 미만이므로 삭제함

call_seoul = call_seoul.dropna()
call_sejong = call_sejong.dropna()
call_suwon = call_suwon.dropna()
call_masan = call_masan.dropna()
call_changwon = call_changwon.dropna()
call_jinhae = call_jinhae.dropna()
call_jinju = call_jinju.dropna()

In [56]:
## 핫덱 대체를 위한 대체군 조회

print(call_seoul[call_seoul['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_sejong[call_sejong['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_suwon[call_suwon['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_masan[call_masan['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_changwon[call_changwon['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_jinhae[call_jinhae['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_jinju[call_jinju['inc_info']=='내용확인불가']['report_sx'].value_counts())

불상    189502
여성    111725
남성     91752
Name: report_sx, dtype: int64
불상    2826
남성     549
여성     378
Name: report_sx, dtype: int64
불상    22094
여성    14961
남성    14868
Name: report_sx, dtype: int64
불상    9920
남성    8880
여성    5105
Name: report_sx, dtype: int64
불상    10353
남성     7710
여성     7710
Name: report_sx, dtype: int64
불상    3677
남성    2584
여성    1934
Name: report_sx, dtype: int64
불상    8535
여성    8325
남성    8016
Name: report_sx, dtype: int64


In [57]:
## 결측치 처리에 참고가 될만한 변수는 성별밖에 없으므로, 내용확인불가 중 성별까지 불상인 데이터는 삭제한다.

seoul = call_seoul[(call_seoul.report_sx == '불상') & (call_seoul.inc_info == '내용확인불가')].index
call_seoul = call_seoul.drop(seoul)

sejong = call_sejong[(call_sejong.report_sx == '불상') & (call_sejong.inc_info == '내용확인불가')].index
call_sejong = call_sejong.drop(sejong)

suwon = call_suwon[(call_suwon.report_sx == '불상') & (call_suwon.inc_info == '내용확인불가')].index
call_suwon = call_suwon.drop(suwon)

masan = call_masan[(call_masan.report_sx == '불상') & (call_masan.inc_info == '내용확인불가')].index
call_masan = call_masan.drop(masan)

changwon = call_changwon[(call_changwon.report_sx == '불상') & (call_changwon.inc_info == '내용확인불가')].index
call_changwon = call_changwon.drop(changwon)

jinhae = call_jinhae[(call_jinhae.report_sx == '불상') & (call_jinhae.inc_info == '내용확인불가')].index
call_jinhae = call_jinhae.drop(jinhae)

jinju = call_jinju[(call_jinju.report_sx == '불상') & (call_jinju.inc_info == '내용확인불가')].index
call_jinju = call_jinju.drop(jinju)

### 효율적인 결측치 처리를 위해 연도 별로 분류

In [58]:
## date 데이터 타입 변경

call_seoul['date'] = call_seoul['date'].astype(str)
call_sejong['date'] = call_sejong['date'].astype(str)
call_suwon['date'] = call_suwon['date'].astype(str)
call_masan['date'] = call_masan['date'].astype(str)
call_changwon['date'] = call_changwon['date'].astype(str)
call_jinhae['date'] = call_jinhae['date'].astype(str)
call_jinju['date'] = call_jinju['date'].astype(str)

In [59]:
## 연도 슬라이싱

for i in range(len(call_seoul['date'].values)):
    call_seoul['date'].values[i] = call_seoul['date'].values[i][0:6]

for i in range(len(call_sejong['date'].values)):
    call_sejong['date'].values[i] = call_sejong['date'].values[i][0:6]

for i in range(len(call_suwon['date'].values)):
    call_suwon['date'].values[i] = call_suwon['date'].values[i][0:6]
    
for i in range(len(call_changwon['date'].values)):
    call_changwon['date'].values[i] = call_changwon['date'].values[i][0:6]
    
for i in range(len(call_jinhae['date'].values)):
    call_jinhae['date'].values[i] = call_jinhae['date'].values[i][0:6]
    
for i in range(len(call_masan['date'].values)):
    call_masan['date'].values[i] = call_masan['date'].values[i][0:6]
    
for i in range(len(call_jinju['date'].values)):
    call_jinju['date'].values[i] = call_jinju['date'].values[i][0:6]

In [60]:
call_seoul['date'] = call_seoul['date'].astype(int)
call_sejong['date'] = call_sejong['date'].astype(int)
call_suwon['date'] = call_suwon['date'].astype(int)
call_masan['date'] = call_masan['date'].astype(int)
call_changwon['date'] = call_changwon['date'].astype(int)
call_jinhae['date'] = call_jinhae['date'].astype(int)
call_jinju['date'] = call_jinju['date'].astype(int)

In [61]:
seoul_18_down = call_seoul[(call_seoul.date >= 201807) & (call_seoul.date <= 201900)]
seoul_18_down['date'] = '2018 하반기'

seoul_19 = call_seoul[(call_seoul.date >= 201900) & (call_seoul.date <= 202000)]
seoul_19['date'] = '2019'

seoul_20 = call_seoul[(call_seoul.date >= 202000) & (call_seoul.date <= 202100)]
seoul_20['date'] = '2020'

call_seoul = pd.concat([seoul_18_down, seoul_19, seoul_19, seoul_20, seoul_20])


sejong_18_down = call_sejong[(call_sejong.date >= 201807) & (call_sejong.date <= 201900)]
sejong_18_down['date'] = '2018 하반기'

sejong_19 = call_sejong[(call_sejong.date >= 201900) & (call_sejong.date <= 202000)]
sejong_19['date'] = '2019'

sejong_20 = call_sejong[(call_sejong.date >= 202000) & (call_sejong.date <= 202100)]
sejong_20['date'] = '2020'

call_sejong = pd.concat([sejong_18_down, sejong_19, sejong_20])


suwon_18_down = call_suwon[(call_suwon.date >= 201807) & (call_suwon.date <= 201900)]
suwon_18_down['date'] = '2018 하반기'

suwon_19 = call_suwon[(call_suwon.date >= 201900) & (call_suwon.date <= 202000)]
suwon_19['date'] = '2019'

suwon_20 = call_suwon[(call_suwon.date >= 202000) & (call_suwon.date <= 202100)]
suwon_20['date'] = '2020'

call_suwon = pd.concat([suwon_18_down, suwon_19, suwon_20])


masan_18_down = call_masan[(call_masan.date >= 201807) & (call_masan.date <= 201900)]
masan_18_down['date'] = '2018 하반기'

masan_19 = call_masan[(call_masan.date >= 201900) & (call_masan.date <= 202000)]
masan_19['date'] = '2019'

masan_20 = call_masan[(call_masan.date >= 202000) & (call_masan.date <= 202100)]
masan_20['date'] = '2020'

call_masan = pd.concat([masan_18_down, masan_19, masan_20])


changwon_18_down = call_changwon[(call_changwon.date >= 201807) & (call_changwon.date <= 201900)]
changwon_18_down['date'] = '2018 하반기'

changwon_19 = call_changwon[(call_changwon.date >= 201900) & (call_changwon.date <= 202000)]
changwon_19['date'] = '2019'

changwon_20 = call_changwon[(call_changwon.date >= 202000) & (call_changwon.date <= 202100)]
changwon_20['date'] = '2020'

call_changwon = pd.concat([changwon_18_down, changwon_19,changwon_20])


jinhae_18_down = call_jinhae[(call_jinhae.date >= 201807) & (call_jinhae.date <= 201900)]
jinhae_18_down['date'] = '2018 하반기'

jinhae_19 = call_jinhae[(call_jinhae.date >= 201900) & (call_jinhae.date <= 202000)]
jinhae_19['date'] = '2019'

jinhae_20 = call_jinhae[(call_jinhae.date >= 202000) & (call_jinhae.date <= 202100)]
jinhae_20['date'] = '2020'

call_jinhae = pd.concat([jinhae_18_down, jinhae_19, jinhae_20])


jinju_18_down = call_jinju[(call_jinju.date >= 201807) & (call_jinju.date <= 201900)]
jinju_18_down['date'] = '2018 하반기'

jinju_19 = call_jinju[(call_jinju.date >= 201900) & (call_jinju.date <= 202000)]
jinju_19['date'] = '2019'

jinju_20 = call_jinju[(call_jinju.date >= 202000) & (call_jinju.date <= 202100)]
jinju_20['date'] = '2020'

call_jinju = pd.concat([jinju_18_down, jinju_19, jinju_20])

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [62]:
call_seoul_songpa = call_seoul[call_seoul['jur_stn'].str.contains('서울송파', na=False)]
call_seoul_gangseo = call_seoul[call_seoul['jur_stn'].str.contains('서울강서', na=False)]
call_seoul_yungdeongpo = call_seoul[call_seoul['jur_stn'].str.contains('서울영등포', na=False)]
call_seoul_gwanak = call_seoul[call_seoul['jur_stn'].str.contains('서울관악', na=False)]
call_seoul_mapo = call_seoul[call_seoul['jur_stn'].str.contains('서울마포', na=False)]
call_seoul_gangnam = call_seoul[call_seoul['jur_stn'].str.contains('서울강남', na=False)]
call_seoul_jungrang = call_seoul[call_seoul['jur_stn'].str.contains('서울중랑', na=False)]
call_seoul_nowon = call_seoul[call_seoul['jur_stn'].str.contains('서울노원', na=False)]
call_seoul_guro = call_seoul[call_seoul['jur_stn'].str.contains('서울구로', na=False)]
call_seoul_gangdong = call_seoul[call_seoul['jur_stn'].str.contains('서울강동', na=False)]
call_seoul_gwangjin = call_seoul[call_seoul['jur_stn'].str.contains('서울광진', na=False)]
call_seoul_dongdaemun = call_seoul[call_seoul['jur_stn'].str.contains('서울동대문', na=False)]
call_seoul_seocho = call_seoul[call_seoul['jur_stn'].str.contains('서울서초', na=False)]
call_seoul_gangbuk = call_seoul[call_seoul['jur_stn'].str.contains('서울강북', na=False)]
call_seoul_yangchun = call_seoul[call_seoul['jur_stn'].str.contains('서울양천', na=False)]
call_seoul_yongsan = call_seoul[call_seoul['jur_stn'].str.contains('서울용산', na=False)]
call_seoul_suseo = call_seoul[call_seoul['jur_stn'].str.contains('서울수서', na=False)]
call_seoul_dongjak = call_seoul[call_seoul['jur_stn'].str.contains('서울동작', na=False)]
call_seoul_seodaemun = call_seoul[call_seoul['jur_stn'].str.contains('서울서대문', na=False)]
call_seoul_geumchun = call_seoul[call_seoul['jur_stn'].str.contains('서울금천', na=False)]
call_seoul_sungdong = call_seoul[call_seoul['jur_stn'].str.contains('서울성동', na=False)]
call_seoul_eunpyeong = call_seoul[call_seoul['jur_stn'].str.contains('서울은평', na=False)]
call_seoul_dobong = call_seoul[call_seoul['jur_stn'].str.contains('서울도봉', na=False)]
call_seoul_jungbu = call_seoul[call_seoul['jur_stn'].str.contains('서울중부', na=False)]
call_seoul_sungbuk = call_seoul[call_seoul['jur_stn'].str.contains('서울성북', na=False)]
call_seoul_jongro = call_seoul[call_seoul['jur_stn'].str.contains('서울종로', na=False)]
call_seoul_seobu = call_seoul[call_seoul['jur_stn'].str.contains('서울서부', na=False)]
call_seoul_hyehwa = call_seoul[call_seoul['jur_stn'].str.contains('서울혜화', na=False)]
call_seoul_jongam = call_seoul[call_seoul['jur_stn'].str.contains('서울종암', na=False)]
call_seoul_namdaemun = call_seoul[call_seoul['jur_stn'].str.contains('서울남대문', na=False)]
call_seoul_bangbae = call_seoul[call_seoul['jur_stn'].str.contains('서울방배', na=False)]

In [64]:
def na_112(df):
    
    ## df의 18년도 하반기 결측치
    down_18_na = df[(df.date=='2018 하반기') & (df.inc_info=='내용확인불가')]
    down_18_na_male = down_18_na[down_18_na.report_sx=='남성']
    down_18_na_female = down_18_na[down_18_na.report_sx=='여성']
    
    ## df의 18년도 하반기 결측치를 제거한 모집단이 될 데이터
    down_18 = df[df.date=='2018 하반기'].drop(down_18_na.index)
    
    ## 성별 분류
    down_18_male = down_18[down_18.report_sx=='남성']
    down_18_female = down_18[down_18.report_sx=='여성']
    down_18_unknown = down_18[down_18.report_sx=='불상']
    
    ## 사건 별 확률 정의
    down_18_male_prob = list((down_18_male.value_counts() / sum(down_18_male.value_counts().values)).values)
    down_18_female_prob = list((down_18_female.value_counts() / sum(down_18_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_down_18_male = np.random.multinomial(len(down_18_na_male), pvals=down_18_male_prob, size=30)

        down_18_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(down_18_male.value_counts('inc_info').index)):
            a = np.mean(multi_down_18_male[:,i])
            down_18_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_18_na_replace_male)) == len(down_18_na_male):
            down_18_na_replace_male = np.round(down_18_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_down_18_female = np.random.multinomial(len(down_18_na_female), pvals=down_18_female_prob, size=30)

        down_18_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(down_18_female.value_counts('inc_info').index)):
            a = np.mean(multi_down_18_female[:,i])
            down_18_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_18_na_replace_female)) == len(down_18_na_female):
            down_18_na_replace_female = np.round(down_18_na_replace_female)
            break

        else:
            continue
            
    ### down_18_na_replace_male, down_18_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(down_18_na_replace_male)):
        for j in range(int(down_18_na_replace_male[i])):
            complete_list_male.append(down_18_male.value_counts().keys()[i][3])
                
    down_18_na_male['inc_info'] = complete_list_male
    complete_down_18_male = pd.concat([down_18_na_male, down_18_male])
    
    complete_list_female = []
    for i in range(len(down_18_na_replace_female)):
        for j in range(int(down_18_na_replace_female[i])):
            complete_list_female.append(down_18_female.value_counts().keys()[i][3])
                
    down_18_na_female['inc_info'] = complete_list_female
    complete_down_18_female = pd.concat([down_18_na_female, down_18_female])
    
    complete_down_18 = pd.concat([complete_down_18_male, complete_down_18_female, down_18_unknown])

    ################################19 상반기############################################
    ## df의 19년도 상반기 결측치
    up_19_na = df[(df.date=='2019') & (df.inc_info=='내용확인불가')]
    up_19_na_male = up_19_na[up_19_na.report_sx=='남성']
    up_19_na_female = up_19_na[up_19_na.report_sx=='여성']
    
    ## df의 19년도 상반기 결측치를 제거한 모집단이 될 데이터
    up_19 = df[df.date=='2019'].drop(up_19_na.index)
    
    ## 성별 분류
    up_19_male = up_19[up_19.report_sx=='남성']
    up_19_female = up_19[up_19.report_sx=='여성']
    up_19_unknown = up_19[up_19.report_sx=='불상']
    
    ## 사건 별 확률 정의
    up_19_male_prob = list((up_19_male.value_counts() / sum(up_19_male.value_counts().values)).values)
    up_19_female_prob = list((up_19_female.value_counts() / sum(up_19_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_up_19_male = np.random.multinomial(len(up_19_na_male), pvals=up_19_male_prob, size=30)

        up_19_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(up_19_male.value_counts('inc_info').index)):
            a = np.mean(multi_up_19_male[:,i])
            up_19_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(up_19_na_replace_male)) == len(up_19_na_male):
            up_19_na_replace_male = np.round(up_19_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_up_19_female = np.random.multinomial(len(up_19_na_female), pvals=up_19_female_prob, size=30)

        up_19_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(up_19_female.value_counts('inc_info').index)):
            a = np.mean(multi_up_19_female[:,i])
            up_19_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(up_19_na_replace_female)) == len(up_19_na_female):
            up_19_na_replace_female = np.round(up_19_na_replace_female)
            break

        else:
            continue
            
    ### up_19_na_replace_male, up_19_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(up_19_na_replace_male)):
        for j in range(int(up_19_na_replace_male[i])):
            complete_list_male.append(up_19_male.value_counts().keys()[i][3])
                
    up_19_na_male['inc_info'] = complete_list_male
    complete_up_19_male = pd.concat([up_19_na_male, up_19_male])
    
    complete_list_female = []
    for i in range(len(up_19_na_replace_female)):
        for j in range(int(up_19_na_replace_female[i])):
            complete_list_female.append(up_19_female.value_counts().keys()[i][3])
                
    up_19_na_female['inc_info'] = complete_list_female
    complete_up_19_female = pd.concat([up_19_na_female, up_19_female])
    
    complete_up_19 = pd.concat([complete_up_19_male, complete_up_19_female, up_19_unknown])

    #############################20 하반기###########################
    ## df의 20년도 하반기 결측치
    down_20_na = df[(df.date=='2020') & (df.inc_info=='내용확인불가')]
    down_20_na_male = down_20_na[down_20_na.report_sx=='남성']
    down_20_na_female = down_20_na[down_20_na.report_sx=='여성']
    
    ## df의 20년도 하반기 결측치를 제거한 모집단이 될 데이터
    down_20 = df[df.date=='2020'].drop(down_20_na.index)
    
    ## 성별 분류
    down_20_male = down_20[down_20.report_sx=='남성']
    down_20_female = down_20[down_20.report_sx=='여성']
    down_20_unknown = down_20[down_20.report_sx=='불상']
    
    ## 사건 별 확률 정의
    down_20_male_prob = list((down_20_male.value_counts() / sum(down_20_male.value_counts().values)).values)
    down_20_female_prob = list((down_20_female.value_counts() / sum(down_20_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_down_20_male = np.random.multinomial(len(down_20_na_male), pvals=down_20_male_prob, size=30)

        down_20_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(down_20_male.value_counts('inc_info').index)):
            a = np.mean(multi_down_20_male[:,i])
            down_20_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_20_na_replace_male)) == len(down_20_na_male):
            down_20_na_replace_male = np.round(down_20_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_down_20_female = np.random.multinomial(len(down_20_na_female), pvals=down_20_female_prob, size=30)

        down_20_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(down_20_female.value_counts('inc_info').index)):
            a = np.mean(multi_down_20_female[:,i])
            down_20_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_20_na_replace_female)) == len(down_20_na_female):
            down_20_na_replace_female = np.round(down_20_na_replace_female)
            break

        else:
            continue
            
    ### down_20_na_replace_male, down_20_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(down_20_na_replace_male)):
        for j in range(int(down_20_na_replace_male[i])):
            complete_list_male.append(down_20_male.value_counts().keys()[i][3])
                
    down_20_na_male['inc_info'] = complete_list_male
    complete_down_20_male = pd.concat([down_20_na_male, down_20_male])
    
    complete_list_female = []
    for i in range(len(down_20_na_replace_female)):
        for j in range(int(down_20_na_replace_female[i])):
            complete_list_female.append(down_20_female.value_counts().keys()[i][3])
                
    down_20_na_female['inc_info'] = complete_list_female
    complete_down_20_female = pd.concat([down_20_na_female, down_20_female])
    
    complete_down_20 = pd.concat([complete_down_20_male, complete_down_20_female, down_20_unknown])
    
    return pd.concat([complete_down_18,complete_up_19,complete_down_20])

In [ ]:
# 서울시 전체 결측치 처리

call_seoul_songpa = na_112(call_seoul_songpa)
call_seoul_gangseo = na_112(call_seoul_gangseo)
call_seoul_yungdeongpo = na_112(call_seoul_yungdeongpo)
call_seoul_gwanak = na_112(call_seoul_gwanak)
call_seoul_mapo = na_112(call_seoul_mapo)
call_seoul_gangnam = na_112(call_seoul_gangnam)
call_seoul_jungrang = na_112(call_seoul_jungrang)
call_seoul_nowon = na_112(call_seoul_nowon)
call_seoul_guro = na_112(call_seoul_guro)
call_seoul_gangdong = na_112(call_seoul_gangdong)
call_seoul_gwangjin = na_112(call_seoul_gwangjin)
call_seoul_dongdaemun =na_112(call_seoul_dongdaemun)
call_seoul_seocho = na_112(call_seoul_seocho)
call_seoul_gangbuk = na_112(call_seoul_gangbuk)
call_seoul_yangchun = na_112(call_seoul_yangchun)
call_seoul_yongsan = na_112(call_seoul_yongsan)
call_seoul_suseo = na_112(call_seoul_suseo)
call_seoul_dongjak = na_112(call_seoul_dongjak)
call_seoul_seodaemun = na_112(call_seoul_seodaemun)
call_seoul_geumchun = na_112(call_seoul_geumchun)
call_seoul_sungdong = na_112(call_seoul_sungdong)
call_seoul_eunpyeong = na_112(call_seoul_eunpyeong)
call_seoul_dobong = na_112(call_seoul_dobong)
call_seoul_jungbu = na_112(call_seoul_jungbu)
call_seoul_sungbuk = na_112(call_seoul_sungbuk)
call_seoul_jongro = na_112(call_seoul_jongro)
call_seoul_seobu = na_112(call_seoul_seobu)
call_seoul_hyehwa = na_112(call_seoul_hyehwa)
call_seoul_jongam = na_112(call_seoul_jongam)
call_seoul_namdaemun = na_112(call_seoul_namdaemun)
call_seoul_bangbae = na_112(call_seoul_bangbae)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
## 세종시 처리

call_sejong = na_112(call_sejong)

In [ ]:
## 수원시 관할서 별 분류

call_suwon_nambu = call_suwon[call_suwon['jur_stn'].str.contains('', na=False)]
call_suwon_jungbu = call_suwon[call_suwon['jur_stn'].str.contains('', na=False)]
call_suwon_seobu = call_suwon[call_suwon['jur_stn'].str.contains('', na=False)]

## 결측치 처리

call_suwon_nambu = na_112(call_suwon_nambu)
call_suwon_jungbu = na_112(call_suwon_jungbu)
call_suwon_seobu = na_112(call_suwon_seobu)

In [ ]:
call_suwon

In [ ]:
## 마산 관할서 별 분류

call_masan_dongbu = call_masan[call_masan['jur_stn'].str.contains('', na=False)]
call_masan_jungbu = call_masan[call_masan['jur_stn'].str.contains('', na=False)]

## 결측치 처리

call_masan_dongbu = na_112(call_masan_dongbu)
call_masan_jungbu = na_112(call_masan_jungbu)

In [ ]:
call_masan

In [ ]:
## 창원 관할서 별 분류

call_changwon_seobu = call_changwon[call_changwon['jur_stn'].str.contains('', na=False)]
call_changwon_jungbu = call_changwon[call_changwon['jur_stn'].str.contains('', na=False)]

## 결측치 처리

call_changwon_seobu = na_112(call_changwon_seobu)
call_changwon_jungbu = na_112(call_changwon_jungbu)

In [ ]:
## 진해시 처리

call_jinhae = na_112(call_jinhae)

In [ ]:
## 진주시 처리

call_jinju = na_112(call_jinju)